In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
penalties = pd.read_csv("../data/processed/penalties.csv")
penalties

,game_id,team_id,opp_id,penalty,player,pos,date,year,week,quarter,...,time_left,down,dist,ref_crew,declined,offsetting,yardage,home,postseason,phase
0,2009_1_TEN_PIT,PIT,TEN,Def_Unnecessary_Roughness,T.Polamalu,SS,2009-09-10,2009,1,1,...,00:53:20,1,10,Bill Leavy,No,No,15,Yes,No,Def
1,2009_1_TEN_PIT,TEN,PIT,Off_Illegal_Formation,D.Stewart,T,2009-09-10,2009,1,1,...,00:49:27,1,10,Bill Leavy,No,No,5,No,No,Off
2,2009_1_TEN_PIT,PIT,TEN,Off_Holding,H.Ward,WR,2009-09-10,2009,1,2,...,00:40:19,2,1,Bill Leavy,No,No,10,Yes,No,Off
3,2009_1_TEN_PIT,PIT,TEN,Off_Holding,M.Starks,T,2009-09-10,2009,1,2,...,00:39:54,2,6,Bill Leavy,No,No,10,Yes,No,Off
4,2009_1_TEN_PIT,PIT,TEN,Def_Pass_Interference,T.Polamalu,SS,2009-09-10,2009,1,2,...,00:35:19,2,7,Bill Leavy,No,Yes,0,Yes,No,Def
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62653,2023_22_SF_KC,SF,KC,Def_Holding,F.Warner,LB,2024-02-11,2023,22,2,...,00:32:00,3,9,Bill Vinovich,Yes,No,0,No,Yes,Def
62654,2023_22_SF_KC,SF,KC,Off_False_Start,A.Banks,OL,2024-02-11,2023,22,3,...,00:28:26,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62655,2023_22_SF_KC,SF,KC,Off_False_Start,B.Aiyuk,WR,2024-02-11,2023,22,5,...,-1:59:55,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62656,2023_22_SF_KC,KC,SF,Def_Holding,T.McDuffie,DB,2024-02-11,2023,22,5,...,-1:59:09,3,13,Bill Vinovich,No,No,5,Yes,Yes,Def


I'm just going to look at ref crews with >1,000 penalties called, so that we have a large enough sample size of penalties called for each ref crew that we are analyzing.

In [3]:
ref_crew_counts = dict(penalties.groupby(['ref_crew']).size())
keep_ref_crews = {ref_crew for ref_crew, cnt in ref_crew_counts.items() if cnt > 1000}

penalties = penalties[penalties.ref_crew.isin(keep_ref_crews)]
penalties

,game_id,team_id,opp_id,penalty,player,pos,date,year,week,quarter,...,time_left,down,dist,ref_crew,declined,offsetting,yardage,home,postseason,phase
0,2009_1_TEN_PIT,PIT,TEN,Def_Unnecessary_Roughness,T.Polamalu,SS,2009-09-10,2009,1,1,...,00:53:20,1,10,Bill Leavy,No,No,15,Yes,No,Def
1,2009_1_TEN_PIT,TEN,PIT,Off_Illegal_Formation,D.Stewart,T,2009-09-10,2009,1,1,...,00:49:27,1,10,Bill Leavy,No,No,5,No,No,Off
2,2009_1_TEN_PIT,PIT,TEN,Off_Holding,H.Ward,WR,2009-09-10,2009,1,2,...,00:40:19,2,1,Bill Leavy,No,No,10,Yes,No,Off
3,2009_1_TEN_PIT,PIT,TEN,Off_Holding,M.Starks,T,2009-09-10,2009,1,2,...,00:39:54,2,6,Bill Leavy,No,No,10,Yes,No,Off
4,2009_1_TEN_PIT,PIT,TEN,Def_Pass_Interference,T.Polamalu,SS,2009-09-10,2009,1,2,...,00:35:19,2,7,Bill Leavy,No,Yes,0,Yes,No,Def
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62653,2023_22_SF_KC,SF,KC,Def_Holding,F.Warner,LB,2024-02-11,2023,22,2,...,00:32:00,3,9,Bill Vinovich,Yes,No,0,No,Yes,Def
62654,2023_22_SF_KC,SF,KC,Off_False_Start,A.Banks,OL,2024-02-11,2023,22,3,...,00:28:26,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62655,2023_22_SF_KC,SF,KC,Off_False_Start,B.Aiyuk,WR,2024-02-11,2023,22,5,...,-1:59:55,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62656,2023_22_SF_KC,KC,SF,Def_Holding,T.McDuffie,DB,2024-02-11,2023,22,5,...,-1:59:09,3,13,Bill Vinovich,No,No,5,Yes,Yes,Def


I'm also just going to look at penalties that have been called >500 times by these ref crews, for the same reason.

In [4]:
from collections import Counter

penalty_counts = Counter(penalties.penalty)
keep_penalties = {penalty for penalty, cnt in penalty_counts.items() if cnt > 500}

penalties = penalties[penalties.penalty.isin(keep_penalties)]
penalties

,game_id,team_id,opp_id,penalty,player,pos,date,year,week,quarter,...,time_left,down,dist,ref_crew,declined,offsetting,yardage,home,postseason,phase
0,2009_1_TEN_PIT,PIT,TEN,Def_Unnecessary_Roughness,T.Polamalu,SS,2009-09-10,2009,1,1,...,00:53:20,1,10,Bill Leavy,No,No,15,Yes,No,Def
1,2009_1_TEN_PIT,TEN,PIT,Off_Illegal_Formation,D.Stewart,T,2009-09-10,2009,1,1,...,00:49:27,1,10,Bill Leavy,No,No,5,No,No,Off
2,2009_1_TEN_PIT,PIT,TEN,Off_Holding,H.Ward,WR,2009-09-10,2009,1,2,...,00:40:19,2,1,Bill Leavy,No,No,10,Yes,No,Off
3,2009_1_TEN_PIT,PIT,TEN,Off_Holding,M.Starks,T,2009-09-10,2009,1,2,...,00:39:54,2,6,Bill Leavy,No,No,10,Yes,No,Off
4,2009_1_TEN_PIT,PIT,TEN,Def_Pass_Interference,T.Polamalu,SS,2009-09-10,2009,1,2,...,00:35:19,2,7,Bill Leavy,No,Yes,0,Yes,No,Def
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62653,2023_22_SF_KC,SF,KC,Def_Holding,F.Warner,LB,2024-02-11,2023,22,2,...,00:32:00,3,9,Bill Vinovich,Yes,No,0,No,Yes,Def
62654,2023_22_SF_KC,SF,KC,Off_False_Start,A.Banks,OL,2024-02-11,2023,22,3,...,00:28:26,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62655,2023_22_SF_KC,SF,KC,Off_False_Start,B.Aiyuk,WR,2024-02-11,2023,22,5,...,-1:59:55,2,10,Bill Vinovich,No,No,5,No,Yes,Off
62656,2023_22_SF_KC,KC,SF,Def_Holding,T.McDuffie,DB,2024-02-11,2023,22,5,...,-1:59:09,3,13,Bill Vinovich,No,No,5,Yes,Yes,Def


In [5]:
games_per_ref = penalties.groupby('ref_crew')['game_id'].nunique()

penalties_by_ref = penalties.groupby(['ref_crew', 'penalty']).size().reset_index(name='count')
penalties_by_ref = penalties_by_ref.pivot(index='ref_crew', columns='penalty', values='count').fillna(0)
penalties_by_ref = penalties_by_ref.div(games_per_ref, axis=0)
penalties_by_ref

penalty,Def_Encroachment,Def_Face_Mask,Def_Holding,Def_Illegal_Contact,Def_Illegal_Use_of_Hands,Def_Neutral_Zone_Infraction,Def_Offside,Def_Pass_Interference,Def_Roughing_the_Passer,Def_Unnecessary_Roughness,...,Off_Illegal_Shift,Off_Ineligible_Downfield_Pass,Off_Intentional_Grounding,Off_Pass_Interference,Off_Unnecessary_Roughness,ST_Delay_of_Game,ST_False_Start,ST_Holding,ST_Illegal_Block_Above_the_Waist,ST_Unnecessary_Roughness
ref_crew,,,,,,,,,,,,,,,,,,,,,
Adrian Hill,0.098765,0.271605,1.209877,0.358025,0.308642,0.456790,0.962963,1.555556,0.567901,0.555556,...,0.197531,0.345679,0.123457,0.358025,0.111111,0.271605,0.197531,0.580247,0.160494,0.148148
Alex Kemp,0.181818,0.363636,1.333333,0.353535,0.222222,0.565657,1.232323,1.353535,0.585859,0.525253,...,0.171717,0.303030,0.161616,0.515152,0.121212,0.252525,0.161616,0.545455,0.343434,0.171717
Bill Leavy,0.296703,0.329670,0.714286,0.406593,0.186813,0.318681,0.912088,0.934066,0.263736,0.406593,...,0.142857,0.043956,0.153846,0.285714,0.054945,0.153846,0.131868,0.692308,0.318681,0.076923
Bill Vinovich,0.235897,0.215385,1.041026,0.266667,0.333333,0.533333,0.753846,1.128205,0.215385,0.569231,...,0.117949,0.158974,0.143590,0.292308,0.102564,0.138462,0.153846,0.466667,0.200000,0.097436
Brad Allen,0.145570,0.291139,1.335443,0.316456,0.417722,0.436709,0.854430,1.126582,0.430380,0.487342,...,0.145570,0.145570,0.126582,0.354430,0.158228,0.164557,0.196203,0.525316,0.202532,0.113924
Brad Rogers,0.153846,0.256410,1.102564,0.346154,0.256410,0.564103,0.807692,1.564103,0.628205,0.410256,...,0.230769,0.461538,0.192308,0.756410,0.166667,0.128205,0.141026,0.641026,0.217949,0.179487
Carl Cheffers,0.183333,0.212500,0.879167,0.279167,0.329167,0.408333,0.866667,1.012500,0.533333,0.500000,...,0.191667,0.200000,0.220833,0.466667,0.158333,0.162500,0.191667,0.925000,0.400000,0.166667
Clay Martin,0.147368,0.294737,1.200000,0.263158,0.463158,0.578947,0.621053,1.315789,0.378947,0.368421,...,0.336842,0.157895,0.136842,0.442105,0.073684,0.189474,0.189474,0.736842,0.221053,0.189474
Clete Blakeman,0.199115,0.230088,1.132743,0.389381,0.265487,0.455752,0.823009,1.084071,0.513274,0.491150,...,0.212389,0.207965,0.194690,0.402655,0.137168,0.159292,0.163717,0.712389,0.300885,0.163717


We now have a table indicating how many times per game each ref crew calls each penalty per game.

Let's examine which crews call which penalties at an unusually high frequency:

In [6]:
from scipy.stats import norm

confidence = 0.95
z_score = norm.ppf(1-(1-confidence)/2)

deviation_from_mean = penalties_by_ref.sub(penalties_by_ref.mean(), axis=1)
significant_deviation = deviation_from_mean > z_score * deviation_from_mean.std()
significant_deviation

penalty,Def_Encroachment,Def_Face_Mask,Def_Holding,Def_Illegal_Contact,Def_Illegal_Use_of_Hands,Def_Neutral_Zone_Infraction,Def_Offside,Def_Pass_Interference,Def_Roughing_the_Passer,Def_Unnecessary_Roughness,...,Off_Illegal_Shift,Off_Ineligible_Downfield_Pass,Off_Intentional_Grounding,Off_Pass_Interference,Off_Unnecessary_Roughness,ST_Delay_of_Game,ST_False_Start,ST_Holding,ST_Illegal_Block_Above_the_Waist,ST_Unnecessary_Roughness
ref_crew,,,,,,,,,,,,,,,,,,,,,
Adrian Hill,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
Alex Kemp,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Bill Leavy,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Bill Vinovich,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Brad Allen,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Brad Rogers,False,False,False,False,False,False,False,True,False,False,...,False,True,False,True,False,False,False,False,False,False
Carl Cheffers,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Clay Martin,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
Clete Blakeman,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
for crew in significant_deviation.index:
    outlier_penalties = significant_deviation.columns[significant_deviation.loc[crew]]
    
    print("Referee Crew:", crew)
    print("Outlier Penalties:", list(outlier_penalties))
    print()

Referee Crew: Adrian Hill
Outlier Penalties: ['Def_Pass_Interference', 'Off_Illegal_Formation', 'ST_Delay_of_Game']

Referee Crew: Alex Kemp
Outlier Penalties: ['Def_Face_Mask', 'Def_Offside']

Referee Crew: Bill Leavy
Outlier Penalties: []

Referee Crew: Bill Vinovich
Outlier Penalties: []

Referee Crew: Brad Allen
Outlier Penalties: []

Referee Crew: Brad Rogers
Outlier Penalties: ['Def_Pass_Interference', 'Off_Ineligible_Downfield_Pass', 'Off_Pass_Interference']

Referee Crew: Carl Cheffers
Outlier Penalties: []

Referee Crew: Clay Martin
Outlier Penalties: ['Def_Illegal_Use_of_Hands', 'Off_Illegal_Shift']

Referee Crew: Clete Blakeman
Outlier Penalties: []

Referee Crew: Craig Wrolstad
Outlier Penalties: []

Referee Crew: Ed Hochuli
Outlier Penalties: []

Referee Crew: Gene Steratore
Outlier Penalties: ['Def_Encroachment']

Referee Crew: Jeff Triplette
Outlier Penalties: ['Off_Unnecessary_Roughness']

Referee Crew: Jerome Boger
Outlier Penalties: []

Referee Crew: John Hussey
Outli

We can see that Land Clark's crew has a lot of outlier penalties, indicating that his crew referees games a lot differently than other crews, and tends to call more penalties.

We can also see that Adrian Hill and Shawn Smith's crews tend to call Defensive Pass Interference more than other crews. Since Defensive Pass Interference can have a large impact on the outcome of drives, it may be the case that these two crews might have a bigger impact on games than other crews.